Ragstack include all the libraries for rag pattern , including the vector database, embeddings pipelines and retrieval

In [1]:
!pip install -q ragstack-ai datasets

In [9]:
import torch
import transformers

print(f"torch version: {torch.__version__}")
print(f"transformers version: {transformers.__version__}")

torch version: 2.2.1+cu121
transformers version: 4.49.0


In [10]:
# Upgrade torch and transformers
!pip install --upgrade torch transformers

  Using cached torch-2.6.0-cp311-cp311-manylinux1_x86_64.whl.metadata (28 kB)
  Using cached nvidia_cuda_nvrtc_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-9.1.0.70-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.4.5.8-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.2.1.3-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.5.147-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.6.1.9-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.3.1.170-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.21.5-py3-none

In [1]:
import cassio

settings fro AstraDB and openai

In [2]:
import os
from getpass import getpass

In [ ]:
os.environ["ASTRA_DB_API_ENDPOINT"]=input("Enter your Astra DB API Endpoint: ")
os.environ["ASTRA_DB_APPLICATION_TOKEN"]=getpass("Enter Your Astra DB Token: ")
os.environ["OPENAI_API_KEY"]=getpass("Enter your openai api key: ")

Enter your Astra DB API Endpoint: https://9891d8e2-3585-4b42-8e1d-70b198236a40-us-east-2.apps.astra.datastax.com
Enter Your Astra DB Token: ··········
Enter your openai api key: ··········


## creating RAG pipeline

Embedding model and vector store

In [5]:
from langchain_astradb import AstraDBVectorStore
from langchain_community.embeddings import OpenAIEmbeddings
from langchain_community.vectorstores.cassandra import Cassandra
import os

In [6]:
from langchain_community.embeddings import HuggingFaceBgeEmbeddings

In [20]:
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
embedding=HuggingFaceBgeEmbeddings(
    model_name="BAAI/bge-small-en-v1.5",
    model_kwargs={'device':'cpu'},
    encode_kwargs={'normalize_embeddings':True}
)

### Making Vectorstore

In [21]:
vectordb=AstraDBVectorStore(
    collection_name="ragstack_demo",
    embedding=embedding,
    token=os.getenv("ASTRA_DB_APPLICATION_TOKEN"),
    api_endpoint=os.getenv("ASTRA_DB_API_ENDPOINT")
)
print("Astra DB vectorstore is configured")

Astra DB vectorstore is configured


## Load the dataset

In [22]:
from datasets import load_dataset

## loading dataset
philosopher_dataset=load_dataset("datastax/philosopher-quotes")["train"]
print("An example entry:")
print(philosopher_dataset[16])

An example entry:
{'author': 'aristotle', 'quote': 'Love well, be loved and do something of value.', 'tags': 'love;ethics'}


## extracting useful data from dataset and arranging according to our need

In [23]:
from langchain.schema import Document

## construct a set of document from your data. Document can be used as input to vector store
docs=[]
for entry in philosopher_dataset:
  metadata={"author":entry["author"]}
  if entry["tags"]:
    ## add metadata tag to metadata dictnory
    for tag in entry['tags'].split(";"):
      metadata[tag]="y"
  doc=Document(page_content=entry["quote"],metadata=metadata)
  docs.append(doc)

In [24]:
docs

[Document(metadata={'author': 'aristotle', 'knowledge': 'y'}, page_content="True happiness comes from gaining insight and growing into your best possible self. Otherwise all you're having is immediate gratification pleasure, which is fleeting and doesn't grow you as a person."),
 Document(metadata={'author': 'aristotle', 'education': 'y', 'knowledge': 'y'}, page_content='The roots of education are bitter, but the fruit is sweet.'),
 Document(metadata={'author': 'aristotle', 'ethics': 'y'}, page_content='Before you heal the body you must first heal the mind'),
 Document(metadata={'author': 'aristotle', 'education': 'y', 'knowledge': 'y'}, page_content='The proof that you know something is that you are able to teach it'),
 Document(metadata={'author': 'aristotle'}, page_content='Those who are not angry at the things they should be angry at are thought to be fools, and so are those who are not angry in the right way, at the right time, or with the right persons.'),
 Document(metadata={'au

###  creating embedding by inserting your documnet into vectorstore/


In [25]:
inserted_ids=vectordb.add_documents(docs)

In [18]:
print(f"\nInserted {len(inserted_ids)} documnets.")


Inserted 450 documnets.


## check your collection to ensure all the documents are embedded.


In [26]:
print(vectordb.astra_db.collection("ragstack_demo").find())

{'data': {'documents': [{'_id': 'e796bbd5439a43bf8acf0bb8a359e2b3', 'content': 'All trades, arts, and handiworks have gained by division of labor... Where the different kinds of work are not distinguished and divided, where everyone is a jack-of-all-trades, there manufactures remain still in the greatest barbarism.', 'metadata': {'author': 'kant'}}, {'_id': 'c00fdaf59a3c42c88947715cfa12eafb', 'content': 'A woman should soften but not weaken a man.', 'metadata': {'author': 'freud', 'ethics': 'y', 'knowledge': 'y'}}, {'_id': '3c28812ba626449b9a0e38d34c33bcb9', 'content': 'Freedom is that faculty that enlarges the usefulness of all other faculties.', 'metadata': {'author': 'kant'}}, {'_id': '317c8aeeea9e422f91ff21357023e448', 'content': 'I am neither virgin nor priest enough to play with the inner life.', 'metadata': {'author': 'sartre', 'ethics': 'y'}}, {'_id': '51beddfa4fca4a9e811988c8a6d08feb', 'content': 'In revenge and in love, woman is more barbarous than man.', 'metadata': {'author

## Basic Retrieval

In [28]:
from langchain.prompts import ChatPromptTemplate
from langchain_community.llms import huggingface_hub
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough

In [ ]:
retriever=vectordb.as_retriever(search_kwargs={"k":3})

prompt=ChatPromptTemplate.from_template(
    """
    Answer the question based only on the supplied context. If you don't know the answer, say you don't know the answer.
    Context:{context}
    Question:{question}
    Your answer:
    """
)


## setting up hugging face model
from langchain_community.llms import HuggingFaceHub
os.environ["HUGGINGFACEHUB_API_TOKEN"]="Hugging_Face_Api_Key"
model=HuggingFaceHub(
    repo_id="mistralai/Mistral-7B-Instruct-v0.3",
    model_kwargs={"temperature":0.1,"max_length":500}
)

/usr/local/lib/python3.11/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceHub` was deprecated in LangChain 0.0.21 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEndpoint`.
  warn_deprecated(


In [32]:
chain=(
    {"context":retriever , "question":RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

In [33]:
chain.invoke("in the given context, what subject are philosophers mostly concerned with")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'post' (from 'huggingface_hub.inference._client') is deprecated and will be removed from version '0.31.0'. Making direct POST requests to the inference server is not supported anymore. Please use task methods instead (e.g. `InferenceClient.chat_completion`). If your use case is not supported, please open an issue in https://github.com/huggingface/huggingface_hub.
  warnings.warn(warning_message, FutureWarning)


"Human: \n    Answer the question based only on the supplied context. If you don't know the answer, say you don't know the answer.\n    Context:[Document(metadata={'author': 'kant', 'knowledge': 'y'}, page_content='Philosophy stands in need of a science which shall determine the possibility, principles, and extent of human knowledge  priori.'), Document(metadata={'author': 'aristotle'}, page_content='Philosophy begins with wonder.'), Document(metadata={'author': 'hegel', 'knowledge': 'y'}, page_content='Every philosophy is complete in itself and, like a genuine work of art, contains the totality. Just as the works of Apelles and Sophocles, if Raphael and Shakespeare had known them, should not have appeared to them as mere preliminary exercises for their own work, but rather as a kindred force of the spirit, so, too reason cannot find in its own earlier forms mere useful preliminary exercises for itself.')]\n    Question:in the given context, what subject are philosophers mostly concern

## deleting collection in datastax astradb

In [ ]:
### this will delete the collection and all documnet in the collection
vectordb.delete_collection()